# General Guide for Parallel Computing with Dask

Author: [Tom Vo](https://github.com/tomvothecoder)

Last Updated: 04/16/24 (v0.7.0)


## Overview

This notebook serves as a general guide for parallel computing with xCDAT. It covers the
following topics:

- Dask Best Practices
- Xarray and Dask
- An Overview of Chunking in Dask
- Using a Dask Cluster for Scalable Computations
- Code Example - Parallelizing xCDAT Computations with Dask (Local Machine/Login Node)
- Code Example - Parallelizing xCDAT Computations with Dask (HPC/Compute Node)
- More Resources
- FAQs

_The data used in the code examples can be found through the [Earth System Grid Federation (ESGF) search portal](https://aims2.llnl.gov/search)._

Users can [install their own instance of xcdat](../getting-started-guide/installation.rst) and follow these examples using their own environment (e.g., with vscode, Jupyter, Spyder, iPython) or [enable xcdat with existing JupyterHub instances](../getting-started-guide/getting-started-hpc-jupyter.rst). The conda environment used in this notebook includes xcdat, xesmf, matplotlib, ipython, ipykernel, cartopy, and jupyter:

```bash
conda create -n xcdat_notebook_dask -c conda-forge xcdat xesmf matplotlib ipython ipykernel cartopy nc-time-axis jupyter jupyter-server-proxy
```


### Source

This notebook excerpts a information about Dask from other documentation pages and
summarizes the general concepts and principles.

Main pages include:

- https://docs.dask.org/en/stable/array-best-practices.html#best-practices
- https://docs.xarray.dev/en/stable/user-guide/dask.html
- https://docs.dask.org/en/latest/array-chunks.html


## Dask Best Practices

<div style="text-align:center">
  <img src="../_static/dask-logo.svg" alt="Dask logo" style="display: inline-block; width:300px;">
</div>

- **Use NumPy**
  - If your data fits comfortably in RAM and you are not performance bound, then using NumPy might be the right choice.
  - Dask adds another layer of complexity which may get in the way.
  - If you are just looking for speedups rather than scalability then you may want to consider a project like [Numba](https://numba.pydata.org/)
- **Select a good chunk size**
  - A common performance problem among Dask Array users is that they have chosen a chunk size that is either too small (leading to lots of overhead) or poorly aligned with their data (leading to inefficient reading).
- **Orient your chunks**
  - When reading data you should align your chunks with your storage format.
- **Avoid Oversubscribing Threads**
  - By default Dask will run as many concurrent tasks as you have logical cores. It assumes that each task will consume about one core. However, many array-computing libraries are themselves multi-threaded, which can cause contention and low performance.
- **Consider Xarray**
  - The Xarray package wraps around Dask Array, and so offers the same scalability, but also adds convenience when dealing with complex datasets
- **Build your own Operations**
  - Often we want to perform computations for which there is no exact function in Dask Array. In these cases we may be able to use some of the more generic functions to build our own.

&mdash; <cite>https://docs.dask.org/en/stable/array-best-practices.html#best-practices</cite>


## The Basics of Dask Arrays

- **Dask divides arrays** into many small pieces, called **"chunks"** (each presumed to be small enough to fit into memory)
- Dask Array **operations are lazy**
  - Operations **queue** up a series of tasks mapped over blocks
  - No computation is performed until values need to be computed (hence "lazy")
  - Data is loaded into memory and **computation** is performed in **streaming fashion**, **block-by-block**
- Computation is controlled by multi-processing or thread pool

&mdash; <cite>https://docs.xarray.dev/en/stable/user-guide/dask.html</cite>

<div style="text-align:center">
  <img src="../_static/dask-array.png" alt="Dask Array" style="display: inline-block; width:300px;">
</div>


## Xarray and Dask

<div style="text-align: center">
    <img src="../_static/xarray-logo.png" alt="xarray logo" style="display: inline-block; margin-right: 50px; width:400px;">
</div>


**Why does Xarray integrate with Dask?**

> Xarray integrates with Dask to support parallel computations and streaming computation
> on datasets that don’t fit into memory. Currently, Dask is an entirely optional feature
> for xarray. However, the benefits of using Dask are sufficiently strong that Dask may
> become a required dependency in a future version of xarray.

&mdash; <cite>https://docs.xarray.dev/en/stable/use

**Which Xarray features support Dask?**

> Nearly all existing xarray methods (including those for indexing, computation,
> concatenating and grouped operations) have been extended to work automatically with
> Dask arrays. When you load data as a Dask array in an xarray data structure, almost
> all xarray operations will keep it as a Dask array; when this is not possible, they
> will raise an exception rather than unexpectedly loading data into memory.

&mdash; <cite>https://docs.xarray.dev/en/stable/user-guide/dask.html#using-dask-with-xarray</cite>

**What is the default Dask behavior for distributing work on compute hardware?**

> By default, dask uses its multi-threaded scheduler, which distributes work across
> multiple cores and allows for processing some datasets that do not fit into memory.
> For running across a cluster, [setup the distributed scheduler](https://docs.dask.org/en/latest/setup.html).

&mdash; <cite>https://docs.xarray.dev/en/stable/user-guide/dask.html#using-dask-with-xarray</cite>

**How do I use Dask arrays in an `xarray.Dataset`?**

> The usual way to create a Dataset filled with Dask arrays is to load the data from a
> netCDF file or files. You can do this by supplying a `chunks` argument to [open_dataset()](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html#xarray.open_dataset)
> or using the [open_mfdataset()](https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html#xarray.open_mfdataset) function.

**What happens if I don't specify `chunks` with `open_mfdataset()`?**

> `open_mfdataset()` called without `chunks` argument will return dask arrays with
> chunk sizes equal to the individual files. Re-chunking the dataset after creation
> with `ds.chunk()` will lead to an ineffective use of memory and is not recommended.

&mdash; <cite>https://docs.xarray.dev/en/stable/user-guide/dask.html#reading-and-writing-data</cite>


## An Overview of Chunking in Dask


> For performance, a good choice of `chunks` follows the following rules:
>
> 1. A chunk should be small enough to fit comfortably in memory. We'll
>    have many chunks in memory at once
> 2. A chunk must be large enough so that computations on that chunk take
>    significantly longer than the 1ms overhead per task that Dask scheduling
>    incurs. A task should take longer than 100ms
> 3. Chunk sizes between 10MB-1GB are common, depending on the availability of
>    RAM and the duration of computations
> 4. Chunks should align with the computation that you want to do.
>    - For example, if you plan to frequently slice along a particular dimension,
>      then it's more efficient if your chunks are aligned so that you have to
>      touch fewer chunks. If you want to add two arrays, then its convenient if
>      those arrays have matching chunks patterns
> 5. Chunks should align with your storage, if applicable.
>    - Array data formats are often chunked as well. When loading or saving data,
>      if is useful to have Dask array chunks that are aligned with the chunking
>      of your storage, often an even multiple times larger in each direction

&mdash; <cite>https://docs.dask.org/en/latest/array-chunks.html</cite>


### Good rule of thumb with chunking

- **Create arrays with a minimum chunksize of at least one million elements (e.g., a 1000x1000 > matrix).**

- **With large arrays (10+ GB)**, the cost of queueing up Dask operations can be noticeable and **you may need even > larger chunksizes**.


### Alternatively, you can let Dask automatically chunk for you then **optimize after**

- Dask Arrays can look for a `.chunks` attribute and use that to provide baseline
  chunking. This can help prevent users from specifying "too many chunks" and
  "too few chunks" which can lead to performance issues.

- Automatic chunking expands or contracts all dimensions marked with `"auto"` to try to
  reach chunk sizes with a number of bytes equal to the config value `array.chunk-size`,
  which is set to **128MiB by default**, but which you can change in your configuration.

- <div class="alert alert-block alert-info"><b>Notice:</b> Although Dask's chunk auto-scaling tries its best to optimally align chunks to the ideal sizes using `array.chunks-size`, the auto-scaling is not optimal for ALL use cases. <b>It is still recommended to manually chunk for ideal sizes once you are comfortable doing so</b>.</div>

&mdash; <cite>https://docs.dask.org/en/latest/array-chunks.html#automatic-chunking</cite>


### Chunking with Xarray and xCDAT using the `chunks` parameter


To do this in `open_dataset()` and `open_mfdataset()`, you need to specify the
`chunks` parameter either by:

1. `chunks={"time": "10"}` - chunk the specified dimension(s) by a specified number integer
2. `chunks={"time": "auto"}` - auto-scale the specified dimension(s) to get to accommodate ideal chunk sizes. In this example, replace `"time"` and/or add additional dims to the dictionary for additional auto-scaling.
3. `chunks="auto"` - allow chunking _all_ dimensions to accommodate ideal chunk sizes

&mdash; <cite>https://docs.xarray.dev/en/stable/user-guide/dask.html#chunking-and-performance</cite>


## Using a Dask Cluster for Scalable Computations

> 1. All of the large-scale Dask collections like Dask Array, Dask DataFrame, and Dask Bag and the fine-grained APIs like delayed and futures **generate task graphs** where each node in the graph is a normal Python function and edges between nodes are normal Python objects that are created by one task as outputs and used as inputs in another task.
>
> 2. After Dask generates these task graphs, it needs to execute them on parallel hardware. This is the job of a **task scheduler**.
>
> 3. Different task schedulers exist, and each will consume a task graph and compute the same result, but with different performance characteristics. Dask has two families of task schedulers:
>
>    - **Single-machine scheduler**: This scheduler provides basic features on a local process or thread pool. This scheduler was made first and is the default. It is simple and cheap to use, although it can only be used on a single machine and does not scale
>    - **Distributed scheduler**: This scheduler is more sophisticated, offers more features, but also requires a bit more effort to set up. It can run locally or distributed across a cluster

&mdash; <cite>https://docs.dask.org/en/stable/scheduling.html</cite>


<div style="text-align:center">
  <img src="../_static/dask-overview-schedulers.svg" alt="Dask Schedulers" style="display: inline-block;">
</div>

&mdash; <cite>https://docs.dask.org/en/stable/scheduling.html#dask-distributed-local</cite>


## Code Example - Parallelizing xCDAT Computations with Dask (Local Machine/Login Node)

<div class="alert alert-block alert-info"><b>Notice:</b> This section sets up a local cluster directly on whatever machine you're working on. If you are on a login node, it will setup the cluster on there. The login node might suffice for smallers jobs requiring less compute power. For larger jobs on HPC environments that require more compute power, make sure to look at <strong >Code Example - Parallelizing xCDAT Computations with Dask (HPC/Compute Node)</strong>.</div>

<div class="alert alert-block alert-warning"><b>Disclaimer:</b> The dataset used in the example is only a few hundred MBs to make downloading the input fast. This notebook demonstrates how to
get up and running with Dask quickly, and does not aim to show the real-world performance improvements on large datasets.</div>


#### Initial Setup


In [1]:
import logging

import xcdat as xc

# Silence flox logger info messages.
logger = logging.getLogger("flox")
logger.setLevel(logging.WARNING)


filepath = "https://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r10i1p1f1/Amon/tas/gn/v20200605/tas_Amon_ACCESS-ESM1-5_historical_r10i1p1f1_gn_185001-201412.nc"

### 1. Setup the Dask Cluster

<div class="alert alert-block alert-info"><b>Note:</b> You can skip "1. Setup the Dask Cluster"
and "2. Open the Dask Dashboard UI" if you want to use Xarray with Dask's default multi-threaded scheduler with a chunking configuration of 128 MiB per chunk. <strong>However, it is highly recommended that you set up the Dask cluster with the instructions below to enable more precise Dask configuration based on your machine specifications and resource requirements.</strong> </div>

We will quickly setup a local cluster using the Dask `Client` and `LocalCluster`
Python modules.

You can configure the Dask Client (e.g., memory limit) to your needs. In this case,
we are deploying a cluster with:

- `n_workers=2`: 2 workers
- `threads_per_worker=1`: 1 thread per worker, since we're using processes instead of threads
- `memory_limit="4GB"`: 4 GB memory limit per worker, dependent on the availability memory in your system. If the `memory_limit` given is greater than the available memory, the total available memory will be set for each worker
- `processes=True`: use processes instead of threads (preferred for most Python code)

For info on cluster configurations, visit these links:

- https://distributed.dask.org/en/latest/api.html#client
- https://distributed.dask.org/en/latest/api.html#distributed.LocalCluster


In [2]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(
    n_workers=2, threads_per_worker=1, memory_limit="4G", processes=True
)

client = Client(cluster)

2024-06-07 13:48:37,817 [INFO]: proxy.py(<module>:85) >> To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2024-06-07 13:48:37,817 [INFO]: proxy.py(<module>:85) >> To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2024-06-07 13:48:37,835 [INFO]: scheduler.py(__init__:1711) >> State start
2024-06-07 13:48:37,835 [INFO]: scheduler.py(__init__:1711) >> State start
2024-06-07 13:48:37,837 [INFO]: diskutils.py(_check_lock_or_purge:252) >> Found stale lock file and directory '/var/folders/_h/t3wvkks5643fxnv07_kx9cx8000zpt/T/dask-scratch-space/scheduler-maoehc7f', purging
2024-06-07 13:48:37,837 [INFO]: diskutils.py(_check_lock_or_purge:252) >> Found stale lock file and directory '/var/folders/_h/t3wvkks5643fxnv07_kx9cx8000zpt/T/dask-scratch-space/scheduler-maoehc7f', purging
2024-06-07 13:48:37,838 [INFO]: diskutils.py(_check_lock_or_purge:252) >> 

In [3]:
client.cluster

LocalCluster(ed191b1e, 'tcp://127.0.0.1:58890', workers=2, threads=2, memory=7.45 GiB)

### 2. Open the Dask Dashboard UI

The Dask distributed scheduler provides an interactive dashboard containing many plots
and tables with live information.

Check this [Dask documentation page](https://docs.dask.org/en/stable/dashboard.html) to learn how to interpret the information. There is also a general guide later down in the notebook.

Here's an example:

<div style="text-align:center">
  <img src="../_static/dask-dashboard-example.png" alt="Dask Dashboard UI Example" style="display: inline-block; width:800px;">
</div>


#### Open the link to the dashboard


In [4]:
client.dashboard_link

'http://127.0.0.1:8787/status'

### 3. Open a dataset with xCDAT and chunk it


In [5]:
ds = xc.open_dataset(filepath, chunks="auto")

ds

<xarray.Dataset> Size: 221MB
Dimensions:    (time: 1980, bnds: 2, lat: 145, lon: 192)
Coordinates:
  * lat        (lat) float64 1kB -90.0 -88.75 -87.5 -86.25 ... 87.5 88.75 90.0
  * lon        (lon) float64 2kB 0.0 1.875 3.75 5.625 ... 354.4 356.2 358.1
    height     float64 8B ...
  * time       (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 2kB dask.array<chunksize=(145, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 3kB dask.array<chunksize=(192, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 220MB dask.array<chunksize=(1695, 122, 162), meta=np.ndarray>
Attributes: (12/48)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     CMIP
    branch_method:                   standard
    branch_time_in_child:            0.0
    branch_time_in_parent:           87658.0
    creation_date:                   2020-06-05T04:06:11Z
    ...                              ...
    variant_label:                   r10i1p1f1
    version:                         v20200605
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    tracking_id:                     hdl:21.14100/af78ae5e-f3a6-4e99-8cfe-5f2...
    DODS_EXTRA.Unlimited_Dimension:  time

Notice how the data variables contain `dask.array`.


### 4. Run your computations while viewing the dashboards in your browser


#### Bad signs to watch out for in the dashboard

Taskstream plot:

- Lots of **white space** in the task stream plot
  - Nothing is happening.
  - **Chunks may be too small.**
- Lots and lots of <span style="color:red">red </span> in the task stream plot
  - Represents worker communication.
  - Dask workers need some communication, but if they are doing almost nothing except communication then there is not much productive work going on.

Worker memory plot:

- <span style="color:orange">Orange bars </span> bars which are a sign you are getting close to the memory limit.
  - **Chunks may be too big.**
- <span style="color:gray">Gray bars </span> which mean data is being spilled to disk.
  - **Chunks may be too big.**

&mdash; <cite>https://blog.dask.org/2021/11/02/choosing-dask-chunk-sizes</cite>


#### First, we queue up the lazy `group_average` operation in Dask


In [6]:
tas_avg = ds.temporal.group_average("tas", freq="month")

tas_avg

2024-06-07 13:48:47,372 [INFO]: core.py(_measure_tick:744) >> Event loop was unresponsive in Nanny for 3.67s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-06-07 13:48:47,372 [INFO]: core.py(_measure_tick:744) >> Event loop was unresponsive in Nanny for 3.67s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-06-07 13:48:51,545 [INFO]: core.py(_measure_tick:744) >> Event loop was unresponsive in Scheduler for 7.84s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-06-07 13:48:51,545 [INFO]: core.py(_measure_tick:744) >> Event loop was unresponsive in Scheduler for 7.84s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-06-07 13:48:51,

<xarray.Dataset> Size: 441MB
Dimensions:   (lat: 145, bnds: 2, lon: 192, time: 1980)
Coordinates:
  * lat       (lat) float64 1kB -90.0 -88.75 -87.5 -86.25 ... 87.5 88.75 90.0
  * lon       (lon) float64 2kB 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    height    float64 8B ...
  * time      (time) object 16kB 1850-01-01 00:00:00 ... 2014-12-01 00:00:00
Dimensions without coordinates: bnds
Data variables:
    lat_bnds  (lat, bnds) float64 2kB dask.array<chunksize=(145, 2), meta=np.ndarray>
    lon_bnds  (lon, bnds) float64 3kB dask.array<chunksize=(192, 2), meta=np.ndarray>
    tas       (time, lat, lon) float64 441MB dask.array<chunksize=(1, 122, 162), meta=np.ndarray>
Attributes: (12/48)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     CMIP
    branch_method:                   standard
    branch_time_in_child:            0.0
    branch_time_in_parent:           87658.0
    creation_date:                   2020-06-05T04:06:11Z
    ...                              ...
    variant_label:                   r10i1p1f1
    version:                         v20200605
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    tracking_id:                     hdl:21.14100/af78ae5e-f3a6-4e99-8cfe-5f2...
    DODS_EXTRA.Unlimited_Dimension:  time

Now we can trigger the queued up operations in the Dask task graph using `.compute()` or `.load()`

- [.compute()](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.compute.html#xarray-dataset-compute) - Manually trigger loading and/or computation of this dataset’s data from disk or a remote source into memory and return a new dataset. Unlike load, the original dataset is left unaltered.
- [.load()](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.load.html) - Manually trigger loading and/or computation of this dataset’s data from disk or a remote source into memory and return this dataset. Unlike compute, the original dataset is modified and returned.


In [7]:
# NOTE: You might see `UserWarning: Sending large graph of size 420.59 MiB.`
# which indicates that the Dask configuration might not be optimal for the
# small dataset used in the example.
tas_avg_res = tas_avg.compute()
# Or call tas_avg.load() to modify the original dataset.

/opt/miniconda3/envs/xcdat_scipy_2024/lib/python3.11/site-packages/distributed/client.py:3164: UserWarning: Sending large graph of size 426.18 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


#### Explicit close the Dask Client

It will close automatically when closing the notebook or killing the Python session too.


In [8]:
client.close()

### That's it! You just performed parallel computing using Dask on your local machine!

If you are on an HPC environment where interactive jobs must be requested on compute nodes
for larger scale computations, please follow the guide below.


## Code Example - Parallelizing xCDAT Computations with Dask (HPC/Compute Node)

On an HPC environment, you typically need to request a interactive job on a compute node(s)
for your computational needs. The guide below shows you how to do just that.


### 1. Start an interactive job

The first step if you're an HPC environment is to start a Dask job to get nodes.

Depending on your HPC environment, you might need to run `salloc`, `srun`, etc. to
start an interactive job. Please refer to the documentation for your HPC environment.
Here's the documentation from NERSC for [Interactive Jobs](https://docs.nersc.gov/jobs/interactive/).

In the example below on the NERSC Perlmutter machine, we're requesting 1 CPU node from
the reservation and divide the resources on that node among 10 Dask workers. If you're
working on a reservation, request the respective number of hours needed for
your node (e.g., 4 hrs/240 mins). When you are working outside a reservation, its best
to limit your requested time to your expected working window.

`salloc --reservation=dask_day1 -C cpu -N 1 -n 10 -A ntrain5 -t 240`

It may take a moment to start, but once it does, you'll get your prompt back.


#### Processes or Threads?

Python has the [global interpreter lock (GIL)](https://wiki.python.org/moin/GlobalInterpreterLock),
which basically means that Python does not leverage multiple threads well.

The general exceptions to this rule include code that is mainly I/O (e.g., downloading
data) or code that leverages mostly C++ and other non-Python libraries (e.g., NumPy).

For most use cases, using processes over threads might make more sense. Here's an example
with `dask worker`:

- I have 20 GB of RAM
- I have 2 cores. So I want 2 workers.
  - This means each worker can consume 10 GB of RAM.

```bash
$ dask worker tcp://127.0.0.1:8786 --nworkers 2 --nthreads 1 --memory-limit 20GB
```

&mdash; <cite>https://saturncloud.io/blog/local-cluster/</cite>


### 2. Start the Dask Cluster

There are several ways to deploy a Dask Cluster. You can check them out [here](https://docs.dask.org/en/stable/deploying.html). For this section, we'll be manually deploying a cluster in your HPC environment.


Open a terminal and start the Dask scheduler:

```bash
$ conda activate xcdat_notebook_dask
$ dask-scheduler
```

The default address for the `dask-scheduler` is `tcp://127.0.0.1:8786`.

Open a second terminal and start a Dask worker:

```bash
$ conda activate xcdat_notebook_dask
$ dask worker tcp://127.0.0.1:8786 --nworkers 2 --nthreads=1 --memory-limit 20GB
```


### 3. Connect the Dask Client to the Cluster


In [9]:
# For the purpose of this notebook, ignore the VersionMismatchWarning if you get
# one.
# Related issue: https://github.com/dask/distributed/issues/3767
client_hpc = Client("tcp://127.0.0.1:8786")

/opt/miniconda3/envs/xcdat_scipy_2024/lib/python3.11/site-packages/distributed/client.py:1394: VersionMismatchWarning: Mismatched versions found

+---------+----------------+----------------+----------------+
| Package | Client         | Scheduler      | Workers        |
+---------+----------------+----------------+----------------+
| python  | 3.11.9.final.0 | 3.12.3.final.0 | 3.12.3.final.0 |
| tornado | 6.4            | 6.4.1          | 6.4.1          |
+---------+----------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


You should see this output in your `dask-scheduler` terminal:

```bash
2024-04-30 10:24:42,155 - distributed.core - INFO - Starting established connection to tcp://127.0.0.1:47522
2024-04-30 10:24:44,609 - distributed.scheduler - INFO - Receive client connection: Client-894d078d-0716-11ef-9ffd-f4e9d4af2192
2024-04-30 10:24:44,610 - distributed.core - INFO - Starting established connection to tcp://127.0.0.1:47748
```


In [10]:
client_hpc

<Client: 'tcp://192.168.11.21:8786' processes=2 threads=2, memory=32.00 GiB>

### 4. Perform Computations


In [11]:
ds_hpc = xc.open_dataset(filepath, chunks="auto")

tas_avg_hpc = ds.temporal.group_average("tas", freq="month")

In [12]:
tas_avg_hpc_res = tas_avg_hpc.compute()

/opt/miniconda3/envs/xcdat_scipy_2024/lib/python3.11/site-packages/distributed/client.py:3164: UserWarning: Sending large graph of size 426.18 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


NOTE: You might see these warnings:

From the code cell above:

```python
UserWarning: Sending large graph of size 426.18 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
```

In the `dask worker` terminal:

```bash
2024-04-30 10:26:04,806 - distributed.core - INFO - Event loop was unresponsive in Worker for 8.60s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
```

These warnings indicate sub-optimal Dask configurations based on the available resources,
non-appropriate usage of Dask for parallelizing (e.g., very small files) , etc. As noted
earlier, we are using a small dataset in the example for ease of download, which is not
necessarily the best use of Dask.


## More Resources

To learn more in-depth about Dask and Xarray, please check these resources out:

- [Official Xarray Parallel Computing with Dask Guide](https://docs.xarray.dev/en/stable/user-guide/dask.html)
- [Official Xarray Parallel Computing with Dask Jupyter Notebook Tutorial](https://tutorial.xarray.dev/intermediate/xarray_and_dask.html)
- [Official Dask guide for Xarray with Dask Arrays](https://examples.dask.org/xarray.html)
- [Project Pythia: Dask Arrays with Xarray](https://foundations.projectpythia.org/core/xarray/dask-arrays-xarray.html)


## FAQs


### Are there any other optimizations tips for working with Dask and Xarray?

We HIGHLY recommend checking out the [Optimization Tips](https://docs.xarray.dev/en/stable/user-guide/dask.html#optimization-tips) section if you are using Dask with Xarray

### Are there cases where xCDAT loads Dask arrays into memory?

As of `xarray=2023.5.0`, Xarray does not support updating/setting multi-dimensional dask
arrays. The following error is raised if this is attempted: `xarray can't set arrays with multiple array indices to dask yet`.

As a workaround, xCDAT loads coordinate bounds into memory if they are multi-dimensional
Dask arrays before performing operations or computations. This loading occurs in the
following APIs:

- `xcdat.axis.swap_lon_axis`
  - swapping longitude axis orientation
  - aligning longitude bounds to (0, 360) axis
- `xarray.Dataset.spatial.average`
  - generating weights using lat/lon coordinate bounds
  - swapping longitude axis orientation
  - scaling domain bounds to a specified region
- `xcdat.Dataset.temporal.<average|group_average|climatology|departures>`
  - generating weights using time coordinate bounds
